In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
%matplotlib inline

Using TensorFlow backend.


In [0]:
train_data_dir = '/content/drive/My Drive/Augmented '

In [8]:
# Pixel values rescaling from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1. / 255, validation_split=0.2)
batch_size = 32
img_width, img_height = 227, 227
# Retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        shuffle = True,
        batch_size=batch_size,
        #class_mode='binary',
        subset = 'training',
        color_mode='grayscale')

validation_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        shuffle = True,
        batch_size=batch_size,
        #class_mode='binary',
        subset = 'validation',
        color_mode='grayscale')

Found 7716 images belonging to 2 classes.
Found 1929 images belonging to 2 classes.


In [0]:
epochs = 20
train_samples = 7716
validation_samples = 1929

In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, LeakyReLU
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam 
import time

model = Sequential()

model.add(Conv2D(96, (5, 5), input_shape=(227,227,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))

model.add(Conv2D(384, (3, 3)))
model.add(Activation('relu'))

model.add(Conv2D(384, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3,3), strides=3))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(256, activation = 'relu'))
model.add(Dense(2048, activation = 'relu'))
#model.add(Dropout(0.5))

model.add(Dense(2))
model.add(Activation('softmax'))

Adam = tf.keras.optimizers.Adam(learning_rate=0.00001, beta_1=0.9, beta_2=0.999, amsgrad=False)
#tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

model.compile(loss='binary_crossentropy',
              optimizer=Adam,
              metrics=['accuracy'],
              )

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 223, 223, 96)      2496      
_________________________________________________________________
activation_5 (Activation)    (None, 223, 223, 96)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 111, 111, 96)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 109, 109, 256)     221440    
_________________________________________________________________
activation_6 (Activation)    (None, 109, 109, 256)     0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 54, 54, 256)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 52, 52, 384)      

In [11]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
# Model saving callback

checkpointer = ModelCheckpoint(filepath='Breast.h5', monitor='val_acc', verbose=1, save_best_only=True)

# Early stopping
early_stopping = EarlyStopping(monitor='val_acc', verbose=1, patience=5)

history = model.fit_generator(
        train_generator,
        steps_per_epoch=32,
        epochs=epochs,
        validation_data = validation_generator,
        validation_steps = 32,
        callbacks=[checkpointer, early_stopping],
        verbose=1)

Epoch 1/20
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
32/32 [==============================] - 982s 31s/step - loss: 0.6746 - acc: 0.5117

Epoch 00001: val_acc improved from -inf to 0.51172, saving model to Breast.h5
32/32 [==============================] - 1849s 58s/step - loss: 0.6858 - acc: 0.5171 - val_loss: 0.6746 - val_acc: 0.5117
Epoch 2/20
32/32 [==============================] - 561s 18s/step - loss: 0.6638 - acc: 0.6562

Epoch 00002: val_acc improved from 0.51172 to 0.65625, saving model to Breast.h5
32/32 [==============================] - 1181s 37s/step - loss: 0.6650 - acc: 0.5664 - val_loss: 0.6638 - val_acc: 0.6562
Epoch 3/20
32/32 [==============================] - 550s 17s/step - loss: 0.6088 - acc: 0.7178

Epoch 00003: val_acc improved from 0.65625 to 0.71777, saving model to Breast.h5
32/32 [==============================] - 1166s 36s/step - loss: 0.6509 - acc: 0.6768 - val_loss: 0.6088 - val_acc: 0.7178
Epoch 4/20
3

In [0]:
model.save("/content/drive/My Drive/Model/Breast.h5")

In [28]:
import cv2
import tensorflow as tf
import math
#from google.colab.patches import cv2_imshow

CATEGORIES = ["cancer", "normal"]


def prepare(filepath):
    #IMG_SIZE = 220 # 50 in txt-based
    img_array = cv2.imread(filepath)
    new_array = cv2.resize(img_array, (227,227))

    return new_array.reshape(-1, 227,227, 1)


model = tf.keras.models.load_model("/content/drive/My Drive/Model/Breast.h5")

prediction = model.predict([prepare('/content/drive/My Drive/PNG/cancer/cancer_02-C_0080_1.RIGHT_MLO.LJPEG.1.png')])
pred_percent = math.floor(prediction[0][0]*100)
print(pred_percent,"%")
print(prediction)  # will be a list in a list.
print(CATEGORIES[int(prediction[0][0])])

100 %
[[1. 0.]
 [1. 0.]
 [1. 0.]]
normal


In [0]:
normal = 2342
cancer = 2367

if(prediction[0][0]>prediction[0][1]):
  result = prediction[0][0]
else:
  result = prediction[0][1]

print(result)

In [0]:
# Pixel values rescaling from [0, 255] to [0, 1] interval
datagen1 = ImageDataGenerator(rescale=1. / 255)
batch_size = 32
train_data_dir1 = '/content/drive/My Drive/dataset1/' 
# Retrieve images and their classes for train and validation sets
train_generator1 = datagen1.flow_from_directory(
        train_data_dir1,
        target_size=(224, 224),
        shuffle = True,
        batch_size=batch_size,
        #class_mode='binary',
        subset = 'training')

In [0]:
import os
import glob
from PIL import Image

image_list = []
i = 0
for filename in glob.glob('/content/drive/My Drive/dataset/NORMAL/*.jpg'): #assuming jpg
    im=filename
    image_list.append(im)
    #print(i)
    i +=1
len(image_list)

In [0]:
from PIL import Image
j=0
for i in range(len(image_list)):
  temp = image_list[i]
  im = Image.open(temp)
  temp = temp.replace("jpg", "png")
  temp = temp.replace("/content/drive/My Drive/dataset/NORMAL/", "")
  temp = "/content/drive/My Drive/PNG/normal/"+temp
  im.save(temp)
  print(j)
  j=j+1

In [0]:
image_list[0]

In [0]:
temp = image_list[0]
temp = temp.replace("jpg", "png")
temp = temp.replace("/content/drive/My Drive/dataset/CANCER/", "")
temp = "/content/drive/My Drive/PNG/cancer/"+temp
temp